In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/home/kirill/Desktop/code_blocks_big.csv', sep='\t')
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,['invite people for the kaggle party']
1,"\ndf_train = pd.read_csv('../input/train.csv'),",['bring in the six packs']
2,"\ndf_train.columns,",['check the decoration']
3,"\ndf_train['SalePrice'].describe(),",['descriptive statistics summary']
4,"\nsns.distplot(df_train['SalePrice']);,",['histogram']


Уникальных тегов:

In [3]:
data['tag'].nunique()

102123

Размер:

In [4]:
data['tag'].shape

(213535,)

### Удаление стоп-слов, стемминг, оставляем по одному комментарию

In [5]:
# from nltk import download
# download('stopwords')

# import nltk
# nltk.download('wordnet')

#import nltk
#nltk.download('punkt')

# import nltk
# nltk.download('averaged_perceptron_tagger')

In [6]:
from nltk.stem.snowball import SnowballStemmer 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [7]:
st_wr = set(stopwords.words('english'))

In [8]:
ex_str = 'kirill eats grass on the floors, long word endings, churches'

lemmatizer thx to 

https://medium.com/@gaurav5430/using-nltk-for-lemmatizing-sentences-c1bfff963258 

(left a clap):

In [9]:
import nltk
from nltk.corpus import wordnet

wnl = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(lemmatizer, sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

lemmatize_sentence(wnl, ex_str)

'kirill eat grass on the floor , long word ending , church'

In [10]:
stemmer = SnowballStemmer('english')
stemmer.stem(ex_str)

'kirill eats grass on the floors, long word endings, church'

**Важно:** если у вас в табличке есть колонка 'Unnamed 0:' или вообще какая то лишняя, то дропайте ее до этого цикла (почему-то это сильно влияет на производительность)

Тут из нескольких комментариев выбирается самый короткий (обосновано тем, что тогда он с большей вероятностью встречается часто ака больше похож на метку). Отсекаются пробелы и всякие пустые метки.

In [11]:
import re
def clear_tag(tag):
    ready_tag = tag.replace('_', ' ')
    ready_tag = re.sub('[^A-Za-z0-9 ]+', '', ready_tag)
    ready_tag = ready_tag.strip()                    # removes spaces in the start and in the end
    ready_tag = ready_tag.replace(' ', '_')
    return ready_tag # replace spaces with "_" and removes other punctuation

In [12]:
from tqdm import tqdm
all_to_del = set(['', "['']", ' ', '  ', "[' ']", '        ', '     ', '    ', '   ', "['', ' ']", '       ', '      ', \
             '             ', '         ', '          ', "']",\
                 "['   ", "['", "[' ", "['  ", "['    ", "['    "])
rows_to_del = []

In [14]:
for i in tqdm(range(data.shape[0])):
    my_string = data.loc[i, 'tag']
    
    if (my_string is not np.nan) and my_string is not None:
        
        min_len = len(data.loc[i, 'tag']) + 10
        min_el = None
        s = set(list(map(str, my_string.split("', '"))))
        
        for el in s:
            
            # getting rid of punctuation, spaces, adding "_"
            el = clear_tag(el)
            
            if el not in all_to_del and el is not None and len(el) < 100:
                
                # getting rid of stop words
                el = [w.lower() for w in el.split() if w.lower() not in st_wr]
                el = ' '.join(el)
                
                # stemming 
                #el = stemmer.stem(el)
                
                # lemmatizing
                el = lemmatize_sentence(wnl, el)
                
                if len(el) < min_len:
                    min_len = len(el)
                    min_el = el

        if min_el is not None:
            data.loc[i, 'tag'] = min_el
        else:
            rows_to_del.append(i)
    else:
        rows_to_del.append(i)

100%|██████████| 213535/213535 [02:43<00:00, 1308.60it/s]


Было:

In [15]:
data.shape

(213535, 2)

In [16]:
data = data.drop(rows_to_del)

Осталось:

In [17]:
data.shape

(174210, 2)

In [18]:
data = data.reset_index()

In [19]:
data.drop(columns=['index'], axis=1, inplace=True)
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,invite_people_for_the_kaggle_party
1,"\ndf_train = pd.read_csv('../input/train.csv'),",bring_in_the_six_packs
2,"\ndf_train.columns,",check_the_decoration
3,"\ndf_train['SalePrice'].describe(),",descriptive_statistics_summary
4,"\nsns.distplot(df_train['SalePrice']);,",histogram


________

### Предобработка текста

In [20]:
data = data[data['tag'] != '']
data = data[data['tag'] != '_']
data = data[data['tag'] != '__']
data = data[data['tag'] != '___']
data = data[data['tag'] != '____']
data = data[data['tag'] != '_____']

Осталось уже:

In [21]:
data.shape

(174009, 2)

_____________

### Режем по длинне метки:

Здесь проверка, чтобы не было копипаста, потому что комментарий длиннее 30 символов совпавший так много раз явно коппаст:

In [22]:
data = data[data['tag'].str.len() < 30]

In [23]:
print(data.shape)
print(data['tag'].nunique())

(111737, 2)
31190


### Оставим какие-то самые популярные метки

Из самых популярных 5к меток выкину варианты длиннее n

In [24]:
data['tag'].nunique()

31190

In [25]:
from collections import Counter

mc = Counter(data['tag']).most_common(5000)

In [26]:
strange_to_del = []

Это работает не очень 

Если уникальных блоков кода с этим комментарием меньше 75 (естественно стоит эту константу пробовать менять), то удалим эту метку тк это скорее всего копипаст:

Так делаем среди самых популярных 5к тегов чтобы не тратить время на совсем не популярные метки

In [27]:
for el in tqdm(mc):
    if data[data['tag']==el[0]]['code_block'].nunique() < 75:
        strange_to_del.append(el[0])

100%|██████████| 5000/5000 [01:08<00:00, 73.48it/s]


In [28]:
for el in tqdm(strange_to_del):
    data = data[data['tag'] != el]

100%|██████████| 4935/4935 [01:50<00:00, 44.85it/s]


Здесь естественно теряется много данных:

In [29]:
print(data.shape)
print(data['tag'].nunique())

(54430, 2)
26255


Заметим, что выкинули 4940 меток из самых популярных 5000, оставим 30 самых популярных:

Ниже

In [31]:
Counter(data['tag']).most_common(30)

[('linear_algebra', 3130),
 ('plot', 2955),
 ('model', 974),
 ('check_your_answer', 608),
 ('create_x', 558),
 ('your_code_here', 466),
 ('preprocessing', 437),
 ('feature_engineering', 428),
 ('just_uncomment_them', 402),
 ('prediction', 400),
 ('load_data', 385),
 ('submission', 380),
 ('check_your_answers', 353),
 ('random_forest', 342),
 ('data_preprocessing', 306),
 ('training', 277),
 ('test', 272),
 ('visualization', 270),
 ('import', 269),
 ('eda', 257),
 ('predict', 255),
 ('data', 219),
 ('train', 207),
 ('data_cleaning', 203),
 ('input', 201),
 ('logistic_regression', 200),
 ('model_building', 191),
 ('xgboost', 184),
 ('exploratory_data_analysis', 184),
 ('data_visualization', 184)]

In [32]:
# all_to_del - задан вначале ноутбука

In [33]:
for to_del in all_to_del:
    data = data[data['tag'] != to_del]

In [34]:
print(data.shape)
print(data['tag'].nunique())

(54430, 2)
26255


In [35]:
num_tags_to_drop = data['tag'].nunique() - 30
num_tags_to_drop

26225

Выкидываю самые непопулярные (nunique - 30) меток:

In [36]:
least_c_tags = set(Counter(data['tag']).most_common()[-num_tags_to_drop:])

In [37]:
for el in tqdm(least_c_tags):
    data = data[data['tag'] != el[0]]

100%|██████████| 26225/26225 [06:18<00:00, 69.23it/s] 


Смотрим что осталось:

In [165]:
print(data.shape)
print(data['tag'].nunique())

(15382, 2)
30


In [42]:
ready_data = \
data.reset_index().drop(columns=['index'], axis=1)
#ready_data = ready_data.drop(columns=['level_0'], axis=1)

In [43]:
Counter(ready_data['tag']).most_common()[:10]

[('linear_algebra', 3130),
 ('plot', 2955),
 ('model', 974),
 ('check_your_answer', 608),
 ('create_x', 558),
 ('your_code_here', 466),
 ('preprocessing', 437),
 ('feature_engineering', 428),
 ('just_uncomment_them', 402),
 ('prediction', 400)]

In [44]:
Counter(ready_data['tag']).most_common()[-10:]

[('predict', 255),
 ('data', 219),
 ('train', 207),
 ('data_cleaning', 203),
 ('input', 201),
 ('logistic_regression', 200),
 ('model_building', 191),
 ('xgboost', 184),
 ('exploratory_data_analysis', 184),
 ('data_visualization', 184)]

На данном этапе все работает очень плохо, поэтому давайте попробуем руками с метками поработать...

In [55]:
ready_data['tag'].unique()

array(['visualization', 'logistic_regression', 'random_forest',
       'linear_algebra', 'plot', 'load_data', 'prediction', 'xgboost',
       'import', 'preprocessing', 'model', 'predict', 'submission',
       'data', 'training', 'feature_engineering',
       'exploratory_data_analysis', 'train', 'test', 'input',
       'data_visualization', 'data_cleaning', 'eda', 'check_your_answer',
       'data_preprocessing', 'model_building', 'create_x',
       'check_your_answers', 'your_code_here', 'just_uncomment_them'],
      dtype=object)

Вариант для лемматизации:

In [60]:
ready_data.loc[ready_data['tag'] == 'check_your_answers', 'tag'] = 'check_your_answer'
ready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'
ready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'
ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model'


jut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == 'your_code_here'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

Вариант для стемминга:

In [171]:
# train, train_model, model_build, model, fit_model
# train_model, fit_model -> train
# 'data_visu' -> visual
# model_building -> model_build

In [170]:
ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'train_model', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'fit_model', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'data_preprocess', 'tag'] = 'preprocess'
ready_data.loc[ready_data['tag'] == 'check_answers', 'tag'] = 'check_answer'
ready_data.loc[ready_data['tag'] == 'model', 'tag'] = 'model_building'
ready_data.loc[ready_data['tag'] == 'data_visu', 'tag'] = 'visual'
ready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model_build'

jut_inds = ready_data[ready_data['tag'] == 'uncomment_them'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

Без всего:

In [52]:
# data -> load_data
# exploratory_data_analysis, data_exploration -> eda
# data_preprocessing -> preprocessing
# training -> train
# check_your_answer, check_your_answers -> check_answer
# model -> model_building
# just_uncomment_them -> trash


##################### maybe
# prediction - test ?

In [172]:
'''ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'
ready_data.loc[ready_data['tag'] == 'data_exploration', 'tag'] = 'eda'
ready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'
ready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'check_your_answer', 'tag'] = 'check_answer'
ready_data.loc[ready_data['tag'] == 'check_your_answers', 'tag'] = 'check_answer'
ready_data.loc[ready_data['tag'] == 'model', 'tag'] = 'model_building'

jut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)'''

"ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'\nready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'\nready_data.loc[ready_data['tag'] == 'data_exploration', 'tag'] = 'eda'\nready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'\nready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'\nready_data.loc[ready_data['tag'] == 'check_your_answer', 'tag'] = 'check_answer'\nready_data.loc[ready_data['tag'] == 'check_your_answers', 'tag'] = 'check_answer'\nready_data.loc[ready_data['tag'] == 'model', 'tag'] = 'model_building'\n\njut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index\nready_data.drop(index=jut_inds, axis=0, inplace=True)"

In [63]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(14629, 2)
22


[('linear_algebra', 3130),
 ('plot', 2955),
 ('model', 1165),
 ('check_your_answer', 961),
 ('preprocessing', 743),
 ('load_data', 604),
 ('create_x', 558),
 ('train', 484),
 ('eda', 441),
 ('feature_engineering', 428),
 ('prediction', 400),
 ('submission', 380),
 ('random_forest', 342),
 ('test', 272),
 ('visualization', 270),
 ('import', 269),
 ('predict', 255),
 ('data_cleaning', 203),
 ('input', 201),
 ('logistic_regression', 200),
 ('xgboost', 184),
 ('data_visualization', 184)]

Слишком много линейной алгебры и др, давайте из них выберем по 800 объектов (не костыль, а эксперимент)

Удаляем unbalance:

In [77]:
todel_mc1 = np.random.choice(
    np.array(ready_data[ready_data['tag'] == 'linear_algebra'].index),
    3130 - 1000,
    replace = False
)
todel_mc2 = np.random.choice(
    np.array(ready_data[ready_data['tag'] == 'plot'].index),
    2955 - 1000,
    replace = False
)

ready_data.drop(index=todel_mc1, inplace=True, axis=0)
ready_data.drop(index=todel_mc2, inplace=True, axis=0)

In [78]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(10210, 2)
22


[('linear_algebra', 1000),
 ('model', 1000),
 ('check_your_answer', 961),
 ('plot', 831),
 ('preprocessing', 743),
 ('load_data', 604),
 ('create_x', 558),
 ('train', 484),
 ('eda', 441),
 ('feature_engineering', 428),
 ('prediction', 400),
 ('submission', 380),
 ('random_forest', 342),
 ('test', 272),
 ('visualization', 270),
 ('import', 269),
 ('predict', 255),
 ('data_cleaning', 203),
 ('input', 201),
 ('logistic_regression', 200),
 ('xgboost', 184),
 ('data_visualization', 184)]

____________

Соотношения train, test, val = 8.5 / 1.5 / 1.5

Поделим нацело:

In [79]:
(ready_data.shape[0] - 2) / 11

928.0

In [80]:
ready_data = ready_data.drop(ready_data.tail(2).index)

In [81]:
ready_data.shape[0] / 11

928.0

То есть размер трейна:

In [84]:
928.0 * 8

7424.0

Стратифайдом делю на трейн, тест, сплит

In [85]:
# https://stackoverflow.com/questions/40829137/stratified-train-validation-test-split-in-scikit-learn

from sklearn.model_selection import StratifiedShuffleSplit
SEED = 42001

split = StratifiedShuffleSplit(n_splits=1, test_size=3/11, random_state=SEED)
for train_index, test_valid_index in split.split(\
                                                 ready_data.drop('tag', axis=1), \
                                                 ready_data['tag']\
                                                ):
    train_set = ready_data.iloc[train_index]
    test_valid_set = ready_data.iloc[test_valid_index]

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
for test_index, valid_index in split2.split(test_valid_set, test_valid_set['tag']):
    test_set = test_valid_set.iloc[test_index]
    valid_set = test_valid_set.iloc[valid_index]

In [86]:
train_set = train_set.reset_index()
test_set = test_set.reset_index()
valid_set = valid_set.reset_index()

______

Далее все '\n' заменяются на '\n\t', а также вначале каждого сниппета приписывается

'def tag():\n\t'

Каждый сниппет записывается в **отдельный файл** с названием номер_итерации.py в каталог трей, тест или валидейшн.

In [87]:
def add_tabs(exmpl):
    new_exmpl = str()
    for i in range(len(exmpl)):
        if (exmpl[i]=='\n'):
            new_exmpl += '\n\t'
        else:
            new_exmpl += exmpl[i]
    return new_exmpl

train:

In [88]:
for i in range(train_set.shape[0]):
        exmpl = train_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(train_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_train/func'+str(i)+'.py', 'w'))

test:

In [89]:
for i in range(test_set.shape[0]):
        exmpl = test_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(test_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_test/func'+str(i)+'.py', 'w'))

valid:

In [90]:
for i in range(valid_set.shape[0]):
        exmpl = valid_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(valid_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_val/func'+str(i)+'.py', 'w'))

In [91]:
len(valid_set)

1392

_______

Предобработка для просто 30 строк кода посложнее, пока ее нет.